In [ ]:
!pip install --upgrade --force-reinstall numpy==1.26.4 gensim==4.3.3 spacy==3.8.7 pandas==2.2.2 requests==2.32.4 thinc==8.3.4

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached gensim-4.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached spacy-3.8.7-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached thinc-8.3.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
  Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached smart_open-7.3.0.post1-py3-none-any.whl.metadata (24 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.13-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_

In [ ]:
!pip install -q gensim

In [ ]:
!pip install -q pyLDAvis

### Importing Neccessary Libraries

In [ ]:
import pandas as pd
import re
import numpy as np

# NLTK libraries
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize


# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')


from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim
import warnings
warnings.filterwarnings('ignore')
# from gensim.models import CoherenceModel
# from gensim import matutils

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

#### Importing the dataset

In [ ]:
df=pd.read_csv("car_5_brands.csv")

In [ ]:
#First five rows of the data
df.head()

,Unnamed: 0,Rating,car_year,brand_name,date,review
0,0,5.0,2018,Audi,2018-07-11,BEST ALL AROUND PURPOSE CROSSOVER SUV I have n...
1,1,5.0,2018,Audi,2018-06-24,Best car This is a wonderful car. The technol...
2,2,5.0,2018,Audi,2018-05-02,Great Buy Do your home work
3,3,5.0,2018,Audi,2017-12-07,Fun Car Great ride. Loaded with technology. St...
4,4,5.0,2018,Audi,2017-10-25,Best luxury SUV w/ perfect comfort/sport balan...


In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.shape

(31938, 6)

### Data Preprocessing

In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31938 entries, 0 to 31937
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rating      31938 non-null  float64
 1   car_year    31938 non-null  int64  
 2   brand_name  31938 non-null  object 
 3   date        31938 non-null  object 
 4   review      31938 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


In [ ]:
# Check for nun values
df.isnull().sum()

,0
Rating,0
car_year,0
brand_name,0
date,0
review,0


In [ ]:
brand_review_pct = df['brand_name'].value_counts(normalize = True).round(2) * 100
brand_review_pct

,proportion
brand_name,
BMW,25.0
Mercedes-Benz,23.0
Lexus,19.0
Audi,19.0
INFINITI,13.0


In [ ]:
# create a Dataframe for the count of reviews of each brand
brand_review_count = df.groupby('brand_name').count()['review'].reset_index()
brand_review_count

,brand_name,review
0,Audi,6043
1,BMW,8060
2,INFINITI,4307
3,Lexus,6153
4,Mercedes-Benz,7375


In [ ]:
# Remove the numbers from the review
df['review'] = df['review'].apply(lambda x: re.sub(r'[^A-Za-z\s]', '', x))

# Convert the reviews to lowercase
df['review'] = df['review'].map(lambda x: x.lower())
df.review

,review
0,best all around purpose crossover suv i have n...
1,best car this is a wonderful car the technolo...
2,great buy do your home work
3,fun car great ride loaded with technology stee...
4,best luxury suv w perfect comfortsport balance...
...,...
31933,one week in but i love it i just got the car ...
31934,a delightful car i have been driving the gla f...
31935,this car is amazing but visibility is poor we ...
31936,steering wheel module when i first bought this...


In [ ]:
# Setting up the stop-words
stop_words = stopwords.words('english')
# stop_words.extend(['])
stop_words.extend(['porsche,' 'mercede','comfortsport', 'mercedes','mercedes-benz', 'honda','toyota','audi', 'benz','bentley','lexus',
                  'nissan','volvo','drive','nt','like','vehicle','infiniti','good','miles','corvette','come','edmund','lotus','diego','snake',
                 'porsche', 'cayman','bought','year','minute','chicago','car','home', 'work','think','suv','people','edmunds',
                  'cabriolet','lexuss','japan','husband','baby','range', 'rover','cadillac','cadillacs','michelin','texas','second',
                   'awsome','one','now', 'take', 'give', 'new','levinson','road','love','sedan','wife','sport','bang','tank',
                   'truck','lemon','imho','pathfinder','infinity','convertible','allroad','conv','bike','ski','grocery','mclass'
                  ,'hardtop','club','hubby','child','zoom','test','etc','brain','ashamed','carmax','alpina','rocketship','great','germany',
                  'autobahn','mercedez'])

In [ ]:
# Lematizing the reviews
def lematized_review(text):
    rev_text = nlp(text)
    # Extract lematized words
    rev_text = ([token.lemma_.lower() for token in rev_text if not token.is_stop and token.text not in stop_words and not token.is_punct and len(token.text) > 3])
    return rev_text

In [ ]:
import spacy
from tqdm import tqdm

# Load spaCy model, disable unused components for speed
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "textcat"])

def process_doc(doc):
    return [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop
        and token.text.lower() not in stop_words
        and not token.is_punct
        and len(token.text) > 3
    ]

# Convert DataFrame column to list
texts = df['review'].astype(str).tolist()

lemmatized_reviews = []
for doc in tqdm(nlp.pipe(texts, batch_size=100, n_process=-1), total=len(texts)):
    lemmatized_reviews.append(process_doc(doc))

# Save results back to DataFrame
df['review_preprocessed'] = lemmatized_reviews


100%|██████████| 31938/31938 [06:54<00:00, 77.10it/s] 


In [ ]:
df['review_preprocessed'].head()

,review_preprocessed
0,"[well, purpose, crossover, see, drive, own, se..."
1,"[good, wonderful, technology, adaptable, comfo..."
2,[]
3,"[ride, load, technology, steering, tight]"
4,"[good, luxury, perfect, balance, happy, choose..."


In [ ]:
clean_brand_review=df['review_preprocessed']

### LDA - Latent Dirichlet Allocation to understand different aspects

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(clean_brand_review)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(review) for review in clean_brand_review]
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1)]


In [ ]:
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from pprint import pprint

# number of topics
num_topics = 6
# Build LDA model
lda_model = LdaMulticore(corpus=corpus, id2word=id2word,
                     num_topics=num_topics, iterations=400)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.024*"seat" + 0.015*"drive" + 0.013*"comfortable" + 0.012*"ride" + '
  '0.012*"system" + 0.011*"interior" + 0.010*"power" + 0.009*"excellent" + '
  '0.008*"look" + 0.007*"sound"'),
 (1,
  '0.023*"problem" + 0.021*"replace" + 0.016*"repair" + 0.013*"time" + '
  '0.012*"warranty" + 0.012*"dealer" + 0.011*"engine" + 0.011*"year" + '
  '0.010*"go" + 0.010*"issue"'),
 (2,
  '0.011*"drive" + 0.011*"tire" + 0.009*"problem" + 0.009*"time" + '
  '0.008*"seat" + 0.007*"dealer" + 0.007*"ride" + 0.007*"noise" + 0.006*"go" + '
  '0.006*"look"'),
 (3,
  '0.019*"drive" + 0.014*"feel" + 0.014*"well" + 0.011*"look" + 0.009*"ride" + '
  '0.009*"good" + 0.008*"speed" + 0.008*"smooth" + 0.007*"power" + '
  '0.007*"engine"'),
 (4,
  '0.027*"drive" + 0.018*"own" + 0.017*"look" + 0.015*"good" + 0.012*"year" + '
  '0.012*"car" + 0.009*"performance" + 0.009*"luxury" + 0.009*"want" + '
  '0.008*"well"'),
 (5,
  '0.013*"problem" + 0.013*"look" + 0.008*"seat" + 0.007*"power" + '
  '0.006*"time" + 0.006*"

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -7.881658470904062


In [ ]:
#Visualize the topic
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus=corpus, dictionary=id2word,sort_topics=False)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.110227 -0.043666       1        1  15.796916
1     -0.190697  0.018654       2        1  17.238903
2     -0.040967 -0.024294       3        1  18.195731
3      0.078206  0.016156       4        1  13.022890
4      0.029009  0.089341       5        1  18.669165
5     -0.043691 -0.066673       6        1   4.735249
6      0.057912  0.010482       7        1  12.341146, topic_info=        Term         Freq         Total Category  logprob  loglift
139  problem  7441.000000   7441.000000  Default  30.0000  30.0000
844  replace  4383.000000   4383.000000  Default  29.0000  29.0000
285     seat  6878.000000   6878.000000  Default  28.0000  28.0000
503   repair  3210.000000   3210.000000  Default  27.0000  27.0000
43      good  6601.000000   6601.000000  Default  26.0000  26.0000
..       ...          ...           ...      ...      ...      ...
13     drive   636.818695  13043.612335   Topic7  -5.2259  -0.9273
141     year   520.728380   5926.235564   Topic7  -5.4271  -0.3397
127     time   509.464124   6615.054407   Topic7  -5.4490  -0.4715
18      need   477.396416   4415.764896   Topic7  -5.5140  -0.1324
139  problem   494.206057   7441.734489   Topic7  -5.4794  -0.6197

[642 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
13036      1  0.927293         \r   
11067      1  0.972077      \r      
11636      1  0.961851              
196        1  0.374274  acceleration
196        2  0.030051  acceleration
...      ...       ...           ...
141        3  0.100064          year
141        4  0.068171          year
141        5  0.375787          year
141        6  0.017043          year
141        7  0.087914          year

[1811 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7])

### Vectorization

In [ ]:
from gensim.models import Word2Vec

w2v_cbow = Word2Vec(
    sentences=clean_brand_review,   # tokenized reviews
    vector_size=100,        # embedding size
    window=5,               # context window
    min_count=5,            # ignore rare words
    workers=4,              # parallel training
    sg=0                    # CBOW (sg=1 = Skip-Gram)
)

In [ ]:
w2v_cbow.save("car_reviews_cbow.model")

In [ ]:
sentence = "The engine feels very powerful and accelerates smoothly on highways."
# Tokenize sentence
tokens = word_tokenize(sentence.lower())

# Keep only tokens in vocab
tokens = [t for t in tokens if t in w2v_cbow.wv.key_to_index]
print(tokens)
print("Similarity", w2v_cbow.wv.similarity(tokens, "engine"))

['engine', 'powerful', 'smoothly']
Similarity [0.7212896  0.2559401  0.16585352]


### Extracting negative and positive sentiment for reviews

In [ ]:
aspects = {
    "Engine/Performance": ["engine", "powerful", "performance", "speed"],
    "Mileage/Fuel": ["mileage", "fuel", "consumption", "efficiency"],
    "Comfort/Interior": ["comfort", "seat", "interior", "space"],
    "Design/Style": ["design", "look", "style", "exterior"],
    "Price/Value": ["price", "cost", "expensive", "worth"],
    "Service/Maintenance": ["service", "maintenance", "support", "parts"]
}


In [ ]:
def get_aspect(sentence, model, aspects):
    tokens=word_tokenize(sentence.lower())
    words = [t for t in tokens if t in w2v_cbow.wv.key_to_index]
    # words = [w.lower() for w in sentence.split()]
    max_sim, best_aspect = 0, None

    for aspect, keywords in aspects.items():
        for keyword in keywords:
            for word in words:
                if word in model.wv and keyword in model.wv:
                    sim = model.wv.similarity(word, keyword)
                    if sim > max_sim:
                        max_sim, best_aspect = sim, aspect
    return best_aspect


In [ ]:
from textblob import TextBlob

def get_sentiment(sentence):
    polarity = TextBlob(sentence).sentiment.polarity
    return "positive" if polarity > 0 else "negative"


In [ ]:
def get_results(brand, review):
  sentences = sent_tokenize(review)
  results = []
  neg_sens=[]
  for sentence in sentences:
      aspect = get_aspect(sentence, model, aspects)
      sentiment = get_sentiment(sentence)
      if sentiment=="negative":
        neg_sens.append((brand, aspect, sentence))
      results.append((brand, aspect, sentiment))
  return results, neg_sens

In [ ]:
df['review'][0]

'best all around purpose crossover suv i have never seen or driven a car like this before  ive owned bmw   and  series porsche  and mercedes sl  and a honda pilot  the sq  is a combination of all of them  its rides very comfortably very stable and quiet sporty and with a lot of power and torque its a get up and go car great to drive anywhere in the city or country  i love the quality and the very advanced technology i feel like im in a spaceship  ive tested the volvo xc it had a very bouncy air suspension and i didnt quite trust their nav system i also test drove the rx  but again the ride was not as smooth and it just didnt absorb the bumps well plus it was quite boring to drive ive never test drove an audi before until my wife wanted to look at the audi q after we bought the q i just fell in love with it and had to have one just like it  the sq is a combination of sports car and comfortable suv very adaptable to all my different driving needs'

In [ ]:
audi_df = df[df['brand_name'] == "Audi"]
lexus_df = df[df['brand_name'] == "Lexus"]
bmw_df = df[df['brand_name'] == "BMW"]
mercedes_df = df[df['brand_name'] == "Mercedes-Benz"]
infinity_dy = df[df['brand_name'] == "INFINITI"]


In [ ]:
dfs = [audi_df, lexus_df, bmw_df, mercedes_df, infinity_dy]
results = []
neg_sens=[]
temp=[]
for i in dfs:
    brand=i['brand_name'].unique()[0]
    print(brand)
    for idx, row in i.iterrows():   # iterate rows
      temp, a=get_results(row[2], row[4])
      results.append(temp)
      if a:
        neg_sens.append(a)
    print("completed one brand")



Audi
completed one brand
Lexus
completed one brand
BMW
completed one brand
Mercedes-Benz
completed one brand
INFINITI
completed one brand


In [ ]:
import pandas as pd

flat_results = [item for sublist in results for item in sublist]
df_result = pd.DataFrame(flat_results, columns=["brand", "aspect", "sentiment"])

summary = df_result.groupby(["brand", "aspect", "sentiment"]).size().unstack(fill_value=0)
summary["total"] = summary.sum(axis=1)
summary["positive_pct"] = (summary.get("positive", 0) / summary["total"]) * 100
summary["negative_pct"] = (summary.get("negative", 0) / summary["total"]) * 100

summary

sentiment                          negative  positive  total  positive_pct  \
brand         aspect                                                         
Audi          Comfort/Interior           87      1234   1321     93.414080   
              Design/Style               56       564    620     90.967742   
              Engine/Performance        189      2378   2567     92.637320   
              Mileage/Fuel               70       471    541     87.060998   
              Price/Value                91       480    571     84.063047   
              Service/Maintenance        90       330    420     78.571429   
BMW           Comfort/Interior          130      1397   1527     91.486575   
              Design/Style               85       744    829     89.746683   
              Engine/Performance        289      3309   3598     91.967760   
              Mileage/Fuel               77       600    677     88.626292   
              Price/Value               134       720    854     84.309133   
              Service/Maintenance       118       443    561     78.966132   
INFINITI      Comfort/Interior           75       961   1036     92.760618   
              Design/Style               32       424    456     92.982456   
              Engine/Performance         98      1559   1657     94.085697   
              Mileage/Fuel               44       341    385     88.571429   
              Price/Value                52       400    452     88.495575   
              Service/Maintenance        65       253    318     79.559748   
Lexus         Comfort/Interior          137      1422   1559     91.212316   
              Design/Style               66       649    715     90.769231   
              Engine/Performance        170      1930   2100     91.904762   
              Mileage/Fuel               71       536    607     88.303130   
              Price/Value                71       532    603     88.225539   
              Service/Maintenance       103       451    554     81.407942   
Mercedes-Benz Comfort/Interior          158      1460   1618     90.234858   
              Design/Style               85       780    865     90.173410   
              Engine/Performance        236      2470   2706     91.278640   
              Mileage/Fuel               97       615    712     86.376404   
              Price/Value               158       631    789     79.974651   
              Service/Maintenance       164       499    663     75.263952   

sentiment                          negative_pct  
brand         aspect                             
Audi          Comfort/Interior         6.585920  
              Design/Style             9.032258  
              Engine/Performance       7.362680  
              Mileage/Fuel            12.939002  
              Price/Value             15.936953  
              Service/Maintenance     21.428571  
BMW           Comfort/Interior         8.513425  
              Design/Style            10.253317  
              Engine/Performance       8.032240  
              Mileage/Fuel            11.373708  
              Price/Value             15.690867  
              Service/Maintenance     21.033868  
INFINITI      Comfort/Interior         7.239382  
              Design/Style             7.017544  
              Engine/Performance       5.914303  
              Mileage/Fuel            11.428571  
              Price/Value             11.504425  
              Service/Maintenance     20.440252  
Lexus         Comfort/Interior         8.787684  
              Design/Style             9.230769  
              Engine/Performance       8.095238  
              Mileage/Fuel            11.696870  
              Price/Value             11.774461  
              Service/Maintenance     18.592058  
Mercedes-Benz Comfort/Interior         9.765142  
              Design/Style             9.826590  
              Engine/Performance       8.721360  
              Mileage/Fuel            13.623596  
   

In [ ]:
summary.to_csv("summary.csv")

In [ ]:
flatten=[]
for item in neg_sens:
    for i in item:
      flatten.append(i)

In [ ]:
negative_reviewsdf = pd.DataFrame(flatten, columns=["brand", "aspect", "negative_review"])

In [ ]:
negative_reviewsdf.head()

,brand,aspect,negative_review
0,Audi,Price/Value,car of my dreams turns out to be a dud ive wan...
1,Audi,Service/Maintenance,nice car horrible service audi service is hor...
2,Audi,Design/Style,lovely vehicle but far too many issues for the...
3,Audi,Price/Value,q t premium recently purchased the q premium ...
4,Audi,Service/Maintenance,very sorry i bought this car the q is great bu...


In [ ]:
# save to CSV
negative_reviewsdf.to_csv("negative_reviews.csv", index=False)